In [3]:
import json


with open('../spider-create-context-intersect/spider_create_context_train_db_id.json') as f:
  train = json.load(f)

with open('../spider-create-context-intersect/spider_create_context_val_db_id.json') as f:
  validation = json.load(f)



In [4]:
train[0]

{'question': 'How many heads of the departments are older than 56 ?',
 'context': 'CREATE TABLE head (age INTEGER)',
 'answer': 'SELECT count(*) FROM head WHERE age  >  56',
 'db_id': 'department_management'}

In [5]:
# def input_text_to_system_and_user_prompt(input_text):
#   system_prompt = "System: " + input_text + " User:"
#   user_prompt = "System: " + input_text + " User:"
#   return system_prompt, user_prompt

# def input_text_to_system_and_user_prompt(input_str):
#     # Remove the [INST] and [/INST] tags
#     clean_str = input_str.replace("[INST]", "").replace("[/INST]", "").replace("Please write me a SQL statement that answers the following question", "Write an SQL query that answers the following") 
#     # Split the cleaned string into two parts: schema and query, based on two newline characters
#     parts = clean_str.rsplit("\n\n", 1)
#     schema = parts[0].strip()
#     query = parts[1].strip()
#     system_prompt = "You are a powerful text-to-SQL model." + schema
#     return system_prompt, query

# print(train[0]['input'])
# a, b = input_text_to_system_and_user_prompt(train[0]['input'])
# print("A:")
# print(a)
# print("B:")
# print(b)


In [6]:
def system_prompt_from_context(context):
    return f"""You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Context:
{context}"""

In [7]:
def convert_to_openai_format(data_point):
    """
    Converts the data to OpenAI format.
    """
    system_prompt = system_prompt_from_context(data_point['context']) 
    return {"messages": [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": data_point['question']
        },
        {
            "role": "assistant",
            "content": data_point['answer']
        }
    ]}
            
    #         {
    #     "system_prompt": system_prompt,
    #     "user_prompt": user_prompt,
    #     "assistant_response": assistant_response
    # }}
   

In [8]:
def convert_dataset(dataset):
    output = []
    for d in dataset:
        oai_format = convert_to_openai_format(d)
        output.append(oai_format)
    return output

In [9]:
converted_train = convert_dataset(train)
converted_validation = convert_dataset(validation)

In [10]:
converted_validation[0]

{'messages': [{'role': 'system',
   'content': 'You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.\n\nYou must output the SQL query that answers the question.\n\n### Context:\nCREATE TABLE singer (Id VARCHAR)'},
  {'role': 'user', 'content': 'How many singers do we have?'},
  {'role': 'assistant', 'content': 'SELECT count(*) FROM singer'}]}

In [11]:
# Save each as a separate file
with open('train_oai_format.json', 'w') as f:
    json.dump(converted_train, f)

with open('validation_oai_format.json', 'w') as f:
    json.dump(converted_validation, f)